In [5]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [112]:
query = 'Bogotá'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=be3c51e34e684878867130512240209&q=Bogotá&days=1&aqi=no&alerts=no'

In [113]:
response = requests.get(url_clima).json()

In [114]:
response

{'location': {'name': 'Bogotá',
  'region': 'Cundinamarca',
  'country': 'Kolumbien',
  'lat': 4.6,
  'lon': -74.08,
  'tz_id': 'America/Bogota',
  'localtime_epoch': 1725289131,
  'localtime': '2024-09-02 09:58'},
 'current': {'last_updated_epoch': 1725288300,
  'last_updated': '2024-09-02 09:45',
  'temp_c': 18.3,
  'temp_f': 64.9,
  'is_day': 1,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
   'code': 1003},
  'wind_mph': 17.4,
  'wind_kph': 28.1,
  'wind_degree': 170,
  'wind_dir': 'S',
  'pressure_mb': 1030.0,
  'pressure_in': 30.42,
  'precip_mm': 0.09,
  'precip_in': 0.0,
  'humidity': 49,
  'cloud': 25,
  'feelslike_c': 18.3,
  'feelslike_f': 64.9,
  'windchill_c': 15.7,
  'windchill_f': 60.2,
  'heatindex_c': 15.7,
  'heatindex_f': 60.2,
  'dewpoint_c': 8.4,
  'dewpoint_f': 47.0,
  'vis_km': 7.0,
  'vis_miles': 4.0,
  'uv': 4.0,
  'gust_mph': 21.9,
  'gust_kph': 35.3},
 'forecast': {'forecastday': [{'date': '2024-09-02',


In [71]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [72]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [73]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [74]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1725246000,
 'time': '2024-09-02 00:00',
 'temp_c': 9.9,
 'temp_f': 49.9,
 'is_day': 0,
 'condition': {'text': 'Clear ',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
  'code': 1000},
 'wind_mph': 2.2,
 'wind_kph': 3.6,
 'wind_degree': 17,
 'wind_dir': 'NNE',
 'pressure_mb': 1019.0,
 'pressure_in': 30.08,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'snow_cm': 0.0,
 'humidity': 84,
 'cloud': 2,
 'feelslike_c': 10.2,
 'feelslike_f': 50.3,
 'windchill_c': 10.2,
 'windchill_f': 50.3,
 'heatindex_c': 9.9,
 'heatindex_f': 49.9,
 'dewpoint_c': 7.4,
 'dewpoint_f': 45.3,
 'will_it_rain': 0,
 'chance_of_rain': 0,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 4.4,
 'gust_kph': 7.1,
 'uv': 0}

In [75]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

'2024-09-02'

In [76]:
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0])

1

In [77]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c']


9.9

In [78]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']


'Clear '

In [79]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [80]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

0

# Dataframe

In [115]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [116]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████| 24/24 [00:00<00:00, 82782.32it/s]


In [102]:
datos[0]

('2024-09-02', 0, 'Clear ', 9.9, 0, 0)

In [117]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2024-09-02,0,Patchy rain nearby,9.9,1,89
1,2024-09-02,1,Patchy rain nearby,10.0,0,66
2,2024-09-02,2,Patchy rain nearby,9.9,0,61
3,2024-09-02,3,Patchy rain nearby,9.6,0,62
4,2024-09-02,4,Patchy rain nearby,9.3,1,89
5,2024-09-02,5,Partly Cloudy,9.2,0,0
6,2024-09-02,6,Partly Cloudy,8.7,0,0
7,2024-09-02,7,Patchy rain nearby,10.8,0,63
8,2024-09-02,8,Patchy rain nearby,13.5,0,61
9,2024-09-02,9,Partly cloudy,18.3,1,100


In [ ]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [118]:
df[df['Lluvia']==1]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2024-09-02,0,Patchy rain nearby,9.9,1,89
4,2024-09-02,4,Patchy rain nearby,9.3,1,89
9,2024-09-02,9,Partly cloudy,18.3,1,100
10,2024-09-02,10,Patchy rain nearby,17.2,1,100
11,2024-09-02,11,Patchy rain nearby,17.8,1,100
12,2024-09-02,12,Patchy rain nearby,17.6,1,100
13,2024-09-02,13,Patchy rain nearby,17.9,1,100
14,2024-09-02,14,Patchy rain nearby,17.5,1,100
15,2024-09-02,15,Patchy rain nearby,16.9,1,100
16,2024-09-02,16,Patchy rain nearby,16.1,1,100


In [122]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [123]:
df['Fecha'][0]

'2024-09-02'

In [124]:
df_rain

,Condicion
Hora,
9,Partly cloudy
10,Patchy rain nearby
11,Patchy rain nearby
12,Patchy rain nearby
13,Patchy rain nearby
14,Patchy rain nearby
15,Patchy rain nearby
16,Patchy rain nearby
17,Patchy rain nearby


In [125]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2024-09-02 en Bogotá es : \n\n\n                Condicion\nHora                    \n9          Partly cloudy\n10    Patchy rain nearby\n11    Patchy rain nearby\n12    Patchy rain nearby\n13    Patchy rain nearby\n14    Patchy rain nearby\n15    Patchy rain nearby\n16    Patchy rain nearby\n17    Patchy rain nearby\n18    Patchy rain nearby\n19    Patchy rain nearby\n20    Patchy rain nearby\n21    Patchy rain nearby'

In [126]:
PHONE_NUMBER

'+14434023042'

# Mensaje SMS desde Twilio

In [127]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+542213596540'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SM5cc3861017d8234f7041b155aaf5bc79
